In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import dgl

In [35]:
# Node file
gse = pd.read_csv('../data/GSE_full.csv', sep=',')

# Edge file
ppi = pd.read_csv('../data/PPI_net.txt', sep=' ')

In [36]:
gse.head(5)

,Unnamed: 0,5'nucleotidase increased,AGEP,AIDS dementia complex,AION,AML progression,ANA increased,Abasia,Abdominal abscess,Abdominal adhesions,...,Xerophthalmia,Xerosis,Yawning,Yeast infection,Yellow nail syndrome,Yellow skin,Zinc deficiency,Zollinger-Ellison syndrome,pH urine decreased,pH urine increased
0,9606.ENSP00000000233,0,1,1,1,0,0,1,0,0,...,10,2,19,0,0,2,0,0,0,0
1,9606.ENSP00000000412,0,2,1,3,1,1,1,1,0,...,12,4,32,0,0,4,0,0,0,1
2,9606.ENSP00000001008,0,2,1,3,1,1,1,1,0,...,16,4,33,0,0,4,0,0,0,1
3,9606.ENSP00000001146,0,0,0,0,0,0,0,0,0,...,1,0,3,0,0,0,0,0,0,0
4,9606.ENSP00000002125,0,0,0,1,0,0,0,0,0,...,3,0,6,0,0,0,0,0,0,0


In [7]:
ppi = ppi[['protein1', 'protein2', 'neighborhood']]
ppi['neighborhood'] = 1
ppi.head(5)

,protein1,protein2,neighborhood
0,9606.ENSP00000000233,9606.ENSP00000272298,1
1,9606.ENSP00000000233,9606.ENSP00000253401,1
2,9606.ENSP00000000233,9606.ENSP00000401445,1
3,9606.ENSP00000000233,9606.ENSP00000418915,1
4,9606.ENSP00000000233,9606.ENSP00000327801,1


In [41]:
#Nodes need to be numerical and ordered corectly, so we make a id dic
genes = gse['Unnamed: 0'].tolist()
gene_ids = {}
for i, gene in enumerate(genes):
    gene_ids[gene] = i+1
gene_ids   

{'9606.ENSP00000000233': 1,
 '9606.ENSP00000000412': 2,
 '9606.ENSP00000001008': 3,
 '9606.ENSP00000001146': 4,
 '9606.ENSP00000002125': 5,
 '9606.ENSP00000002165': 6,
 '9606.ENSP00000002596': 7,
 '9606.ENSP00000002829': 8,
 '9606.ENSP00000003084': 9,
 '9606.ENSP00000003100': 10,
 '9606.ENSP00000003302': 11,
 '9606.ENSP00000004531': 12,
 '9606.ENSP00000005178': 13,
 '9606.ENSP00000005226': 14,
 '9606.ENSP00000005257': 15,
 '9606.ENSP00000005260': 16,
 '9606.ENSP00000005284': 17,
 '9606.ENSP00000005286': 18,
 '9606.ENSP00000005340': 19,
 '9606.ENSP00000005386': 20,
 '9606.ENSP00000005587': 21,
 '9606.ENSP00000005995': 22,
 '9606.ENSP00000006015': 23,
 '9606.ENSP00000006053': 24,
 '9606.ENSP00000006275': 25,
 '9606.ENSP00000006526': 26,
 '9606.ENSP00000006658': 27,
 '9606.ENSP00000006724': 28,
 '9606.ENSP00000006777': 29,
 '9606.ENSP00000007390': 30,
 '9606.ENSP00000007414': 31,
 '9606.ENSP00000007699': 32,
 '9606.ENSP00000007722': 33,
 '9606.ENSP00000007735': 34,
 '9606.ENSP00000008391'

In [10]:
#There are 11 genes that are in GSE, so we remove them.
drop_list =[] 

for gene in ppi['protein1'].unique().tolist():
    if gene not in gene_ids:
        drop_list.append(gene)
    else: pass
drop_list

for gene in drop_list: 
    ppi.drop(ppi.loc[ppi['protein1'] == gene].index, inplace=True)
    ppi.drop(ppi.loc[ppi['protein2'] == gene].index, inplace=True)


In [11]:
drop_list

['9606.ENSP00000324464',
 '9606.ENSP00000346484',
 '9606.ENSP00000349458',
 '9606.ENSP00000369126',
 '9606.ENSP00000380044',
 '9606.ENSP00000384262',
 '9606.ENSP00000421689',
 '9606.ENSP00000435460',
 '9606.ENSP00000470087']

In [12]:
#converting to gene ids
#%%time
def gene2id (gene):
    return(gene_ids[gene])

gse['Unnamed: 0'] = gse.apply(lambda x: gene2id(x['Unnamed: 0']), axis=1)
ppi['protein1'] = ppi.apply(lambda x: gene2id(x['protein1']), axis=1)
ppi['protein2'] = ppi.apply(lambda x: gene2id(x['protein2']), axis=1)

In [14]:
ppi.to_csv('../data/GNN-PPI-net.csv', index=False, sep = ',')
ppi

,protein1,protein2,neighborhood
0,1,3469,1
1,1,1759,1
2,1,15624,1
3,1,16446,1
4,1,7409,1
...,...,...,...
11759449,19555,6049,1
11759450,19555,8711,1
11759451,19555,9472,1
11759452,19555,12490,1


In [17]:
PSE = pd.read_csv('../data/TWOSIDE-train-PSE-964.csv', sep = ',')
PSE

,Drug1,Drug2,PSE
0,Temazepam,sildenafil,Arthralgia
1,Bumetanide,Oxytocin,Arthralgia
2,Tamoxifen,Prednisone,Diarrhoea
3,Temazepam,sildenafil,Diarrhoea
4,Bumetanide,Oxytocin,Diarrhoea
...,...,...,...
18492593,Ketoprofen,montelukast,Pulmonary embolism
18492594,Ketoprofen,montelukast,Cough
18492595,Ketoprofen,montelukast,Rash
18492596,Ketoprofen,montelukast,Alopecia


In [22]:
PSEs = PSE['PSE'].unique().tolist()
PSEs = ['Unnamed: 0'] + PSEs
PSEs

['Unnamed: 0',
 'Arthralgia',
 'Diarrhoea',
 'Headache',
 'Vomiting',
 'Dyspepsia',
 'Renal impairment',
 'Cough',
 'Rhinorrhoea',
 'Malaise',
 'Back pain',
 'Fluid retention',
 'Oedema peripheral',
 'Oedema',
 'Death',
 'Cerebrovascular accident',
 'Pain',
 'Wound',
 'Fatigue',
 'Gastrooesophageal reflux disease',
 'Sinusitis',
 'Urinary tract infection',
 'Fall',
 'Bone pain',
 'Nausea',
 'Paraesthesia',
 'General physical health deterioration',
 'Abdominal pain',
 'Anxiety',
 'Emotional distress',
 'Injury',
 'Memory impairment',
 'Rash',
 'Alanine aminotransferase increased',
 'Drug eruption',
 'Chest pain',
 'Dyspnoea',
 'Stress',
 'Asthenia',
 'Depression',
 'Dysgeusia',
 'Weight decreased',
 'Pruritus',
 'Feeling abnormal',
 'Muscle spasms',
 'Nightmare',
 'Convulsion',
 'Hyperhidrosis',
 'Urinary retention',
 'Dizziness',
 'Haemoglobin decreased',
 'White blood cell count decreased',
 'Burning sensation',
 'Flushing',
 'Anhedonia',
 'Gastrointestinal haemorrhage',
 'Depressed m

In [24]:
gse[PSEs].to_csv('../data/GNN-GSE_full-.csv', index=False, sep = ',')
gse[PSEs]

,Unnamed: 0,Arthralgia,Diarrhoea,Headache,Vomiting,Dyspepsia,Renal impairment,Cough,Rhinorrhoea,Malaise,...,Hyperventilation,Pleuritic pain,Nail disorder,Aneurysm,Thrombophlebitis,Hypertensive crisis,Pneumonia bacterial,Hepatocellular injury,Shock haemorrhagic,Haemorrhagic stroke
0,1,217,332,377,349,209,54,175,46,164,...,34,9,25,3,64,13,2,25,0,7
1,2,336,564,638,608,350,98,286,71,281,...,58,14,48,5,115,21,3,49,0,10
2,3,358,601,680,648,372,104,304,75,300,...,62,15,52,5,122,22,3,55,1,11
3,4,16,26,33,29,17,3,21,7,16,...,5,0,3,1,8,1,0,1,0,1
4,5,37,58,68,66,41,9,37,11,30,...,8,1,7,1,12,3,0,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19550,19551,146,210,246,225,142,38,121,32,100,...,19,5,20,3,36,8,0,13,0,7
19551,19552,3,5,7,5,2,0,5,1,2,...,0,0,0,0,0,0,0,0,0,0
19552,19553,5,9,11,9,4,0,7,1,4,...,1,0,2,0,3,0,0,0,0,0
19553,19554,584,977,1114,1087,606,168,509,121,490,...,90,23,75,12,206,32,5,85,1,17


### Making drugs specific graphs

In [31]:
# Node file
nodes = pd.read_csv('../data/GNN-GSE_full.csv', sep=',')

# Edge file
edges = pd.read_csv('../data/GNN-PPI-net.csv', sep=',')

# Drug-protein file (DTI)
dti = pd.read_csv('../data/GNN-DTI.csv', sep=',')

In [30]:
nodes

,proteinID,Arthralgia,Diarrhoea,Headache,Vomiting,Dyspepsia,Renal impairment,Cough,Rhinorrhoea,Malaise,...,Hyperventilation,Pleuritic pain,Nail disorder,Aneurysm,Thrombophlebitis,Hypertensive crisis,Pneumonia bacterial,Hepatocellular injury,Shock haemorrhagic,Haemorrhagic stroke
0,1,217,332,377,349,209,54,175,46,164,...,34,9,25,3,64,13,2,25,0,7
1,2,336,564,638,608,350,98,286,71,281,...,58,14,48,5,115,21,3,49,0,10
2,3,358,601,680,648,372,104,304,75,300,...,62,15,52,5,122,22,3,55,1,11
3,4,16,26,33,29,17,3,21,7,16,...,5,0,3,1,8,1,0,1,0,1
4,5,37,58,68,66,41,9,37,11,30,...,8,1,7,1,12,3,0,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19550,19551,146,210,246,225,142,38,121,32,100,...,19,5,20,3,36,8,0,13,0,7
19551,19552,3,5,7,5,2,0,5,1,2,...,0,0,0,0,0,0,0,0,0,0
19552,19553,5,9,11,9,4,0,7,1,4,...,1,0,2,0,3,0,0,0,0,0
19553,19554,584,977,1114,1087,606,168,509,121,490,...,90,23,75,12,206,32,5,85,1,17


In [7]:
edges

,protein1,protein2,neighborhood
0,1,3469,1
1,1,1759,1
2,1,15624,1
3,1,16446,1
4,1,7409,1
...,...,...,...
11738325,19555,6049,1
11738326,19555,8711,1
11738327,19555,9472,1
11738328,19555,12490,1


In [45]:
dti = dti.rename(columns={'variable': 'DrugID', 'GeneID': 'ProteinID','value': 'pKd' })
dti['ProteinID'] = dti.apply(lambda x: gene2id(x['ProteinID']), axis=1)
dti = dti[['DrugID', 'ProteinID']]
dti

,DrugID,ProteinID
0,85,32
1,85,90
2,85,100
3,85,119
4,85,153
...,...,...
8914333,71306834,19532
8914334,71306834,19535
8914335,71306834,19539
8914336,71306834,19544


In [ ]:
def drug2ppi(drug):
    

In [19]:
#introducing edges
src = ppi['protein1'].to_numpy()
dst = ppi['protein2'].to_numpy()

# Create a DGL graph from a pair of numpy arrays
g = dgl.graph((src, dst))

# Print a graph gives some meta information such as number of nodes and edges.
print(g)

Graph(num_nodes=19556, num_edges=11738330,
      ndata_schemes={}
      edata_schemes={})


In [20]:
print('#Nodes', g.number_of_nodes())
print('#Edges', g.number_of_edges())

#Nodes 19556
#Edges 11738330


In [ ]:
%%time
# Since the actual graph is undirected, we convert it for visualization purpose.
nx_g = g.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_g)
nx.draw(nx_g, pos, with_labels=True, node_color=[[.7, .7, .7]])